In [ ]:
import sys
import traceback

from snowflake.snowpark.context import get_active_session


session = get_active_session()

notebook_name = "python_test_notebook"
output_file_name = f"{notebook_name}_output.txt"
snowlfake_core_zip_file_name = "snowflake_core.zip"
stage_name = "STAGE_PYTHON_TEST_NOTEBOOK"
pytest_args = ["/tmp/expanded/snowflake_core/tests/integ/", "-W", "ignore", "-r", "a", "--no-cov", "-v"]
is_gov = "GOV" in session.sql("SELECT CURRENT_REGION() as region").first()["REGION"]
if is_gov:
    pytest_args.extend(["-m", "not flaky and not skip_gov"])
else:
    pytest_args.extend(["-m", "not flaky"])

with open(output_file_name, "w") as f:
    sys.stdout = f
    sys.stderr = f
    try:
        import os
        import zipfile

        import pytest

        os.environ["RUNNING_IN_NOTEBOOK"] = "True"
        os.environ["PYTHONUNBUFFERED"] = "1"
        should_enable_logging = os.environ.get("ENABLE_DEBUG_LOGGING_IN_SNOWPY") == "True"

        def clear_modules(prefix):
            """Remove modules with the specified prefix from sys.modules."""
            for module_name in list(sys.modules.keys()):
                if module_name.startswith(prefix) and (module_name != "snowflake.connector.nanoarrow_arrow_iterator"):
                    del sys.modules[module_name]

        clear_modules("snowflake.core")

        get_status = session.file.get(f"@{stage_name}/zip/{snowlfake_core_zip_file_name}", "/tmp/zip")
        if len(get_status) != 1:
            raise Exception("not able to load the snowflake_core")

        with zipfile.ZipFile(f"/tmp/zip/{snowlfake_core_zip_file_name}", "r") as zip_ref:
            zip_ref.extractall("/tmp/expanded/snowflake_core")
        sys.path.insert(0, "/tmp/expanded/snowflake_core/src")
        print("Successfully completed the set up for executing the integration test in notebook")
        try:
            if should_enable_logging:
                import logging
                import pathlib

                import snowflake.core

                snowflake.core.simple_file_logging(
                    path=pathlib.Path("execution_output.txt"),
                    level=logging.DEBUG,
                    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
                )
            exit_code = pytest.main(pytest_args)
            if exit_code != 0:
                print("Some tests failed")
            else:
                print("All tests passed")
            print("Successfully executed integration test in notebook")
        except Exception:
            traceback.print_exc(file=f)
            print("Failed to execute integration test in notebook")
    except Exception:
        traceback.print_exc(file=f)
        print("Failed to complete the set up for executing the integration test in notebook")


put_result = session.file.put(output_file_name, f"@{stage_name}/{notebook_name}", auto_compress=False)